In [3]:
import sys
from pathlib import Path

from openai.types.chat.chat_completion import ChatCompletion

sys.path.append(str(Path(".").resolve().parent))

from config.openai_client import client

# Запрос в Chat

Дока для API: https://platform.openai.com/docs/api-reference/chat

In [18]:
text: str = "What is this 'ML'?"
response: ChatCompletion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": text}],
    max_tokens=1024,
    temperature=0.5,
    # logprobs=True,
)

In [19]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='ML typically stands for Machine Learning, which is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data without being explicitly programmed. Machine learning is used in a wide range of applications, including image and speech recognition, recommendation systems, and autonomous vehicles.', role='assistant', function_call=None, tool_calls=None))

In [20]:
response.choices[0].message.content

'ML typically stands for Machine Learning, which is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data without being explicitly programmed. Machine learning is used in a wide range of applications, including image and speech recognition, recommendation systems, and autonomous vehicles.'